In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd


file_names = []
for i in range(1, 15):  # Adjust the range based on your files
    batch_filenames = np.load(f'/content/drive/MyDrive/MSc Thesis Data Science/influencer_brands/Data/images/filenames_batch{i}.npy')
    file_names.extend(batch_filenames)

data = pd.read_csv('/content/drive/MyDrive/MSc Thesis Data Science/influencer_brands/Data/Metadata/all_metadata_5.csv', lineterminator='\n')

In [7]:
print(data['Sponsorship label'].head)

<bound method NDFrame.head of 0         0
1         0
2         0
3         0
4         0
         ..
196174    0
196175    0
196176    0
196177    0
196178    0
Name: Sponsorship label, Length: 196179, dtype: int64>


Step 1: group and split the data by post_id

In [8]:
from sklearn.model_selection import train_test_split

# Group data by 'post_id'
grouped = data.groupby('post_id')

# Extract the first 'Sponsorship label' for each 'post_id'
sponsorship_labels = grouped['Sponsorship label'].first()

# Create a list of unique post IDs
post_ids = pd.Series(grouped.size().index)

# Split the data with stratification
train_ids, temp_ids = train_test_split(post_ids, test_size=0.2, random_state=42, stratify=sponsorship_labels)
val_ids, test_ids = train_test_split(temp_ids, test_size=0.5, random_state=42, stratify=sponsorship_labels[temp_ids])

In [9]:
print(train_ids.shape)

(150300,)


Step 2: unnest the grouped data

In [10]:
# Unnesting the data
train_data = data[data['post_id'].isin(train_ids)]
val_data = data[data['post_id'].isin(val_ids)]
test_data = data[data['post_id'].isin(test_ids)]

In [11]:
print(f'Train data shape{train_data.shape}')
print(f'Validation data shape{val_data.shape}')
print(f'Test data shape{test_data.shape}')

Train data shape(157038, 33)
Validation data shape(19584, 33)
Test data shape(19557, 33)


Step 3: let's get the image names to match with the image data

In [12]:
# Extract filenames from training and validation datasets
train_filenames = train_data['Image files'].tolist()
val_filenames = val_data['Image files'].tolist()

In [16]:
print(len(train_filenames))
print(len(val_filenames))

157038
19584


Step 3: Match Images to each Dataset

In [17]:
def filter_images_by_filenames(image_batch, filenames_batch, relevant_filenames):
    # Filter both the images and filenames
    filtered_images = []
    for img, fname in zip(image_batch, filenames_batch):
        if fname in relevant_filenames:
            filtered_images.append(img)
    return np.array(filtered_images)

# Assuming you load each image batch and filename batch in a loop or manually
# For example, for batch 1
image_batch1 = np.load('/content/drive/MyDrive/MSc Thesis Data Science/influencer_brands/Data/images/data120_batch1.npy')
filenames_batch1 = np.load('/content/drive/MyDrive/MSc Thesis Data Science/influencer_brands/Data/images/filenames_batch1.npy')

# Filter images for training and validation sets
train_images_batch1 = filter_images_by_filenames(image_batch1, filenames_batch1, train_filenames)
val_images_batch1 = filter_images_by_filenames(image_batch1, filenames_batch1, val_filenames)

In [19]:
print(train_images_batch1.shape)
print(image_batch1.shape)

(11928, 224, 224, 3)
(15000, 224, 224, 3)
